# Construction of a toy model

Import related packages

In [1]:
import cobra
import sys
import tempfile
sys.path.append(r'./code/')
from cobra.io import load_json_model, write_sbml_model, validate_sbml_model
from pprint import pprint
from libsbml import *
from cobra import Model, Metabolite, Reaction
from cobra.flux_analysis import flux_variability_analysis
cobra_config = cobra.Configuration()

Input and output files

In [2]:

# Output files
model_xml_file="./data/models/toy_model.xml"
model_json_file="./data/models/toy_model.json"

In [3]:
# Create a new metabolic model
model = cobra.Model("toy_model")

Add metabolites and reactions

In [4]:
# Add Metabolites

glc__D_c = Metabolite(id='glc__D_c', name='glucose', formula='C6H12O6', compartment='c')
glc__D_e = Metabolite(id='glc__D_e', name='glucose', formula='C6H12O6', compartment='e')
g6p_c = Metabolite(id='g6p_c', name='glucose-6-phosphate', formula='C6H11O9P', compartment='c')
pep_c = Metabolite(id='pep_c', name='phosphoenolpyruvate', formula='C3H2O6P', compartment='c')
pyr_c = Metabolite(id='pyr_c', name='pyruvate', formula='C3H4O3', compartment='c')
f6p_c = Metabolite(id='f6p_c', name='fructose-6-phosphate', formula='C6H11O9P', compartment='c')
adp_c = Metabolite(id='adp_c', name='adenosine diphosphate', formula='C10H15N5O10P2', compartment='c')
atp_c = Metabolite(id='atp_c', name='adenosine triphosphate', formula='C10H12N5O13P3', compartment='c')
fdp_c = Metabolite(id='fdp_c', name='fructose-1,6-bisphosphate', formula='C6H11O13P2', compartment='c')
h_c = Metabolite(id='h_c', name='H+', formula='H', compartment='c')
dhap_c = Metabolite(id='dhap_c', name='dihydroxyacetone phosphate', formula='C3H5O6P', compartment='c')
g3p_c = Metabolite(id='g3p_c', name='glyceraldehyde-3-phosphate', formula='C3H6O6P', compartment='c')
h2o_c = Metabolite(id='h2o_c', name='Water', compartment='c')
_6pgl_c = Metabolite(id='6pgl_c', name='6-phospho-D-glucono-1,5-lactone', formula='C6H10O8P', compartment='c')
_6pgc_c = Metabolite(id='6pgc_c', name='6-phospho-D-gluconate', formula='C6H10O8P', compartment='c')
nadp_c = Metabolite(id='nadp_c', name='nicotinamide adenine dinucleotide phosphate', formula='C21H26N7O17P3', compartment='c')
nadph_c = Metabolite(id='nadph_c', name='reduced nicotinamide adenine dinucleotide phosphate', formula='C21H25N7O14P2', compartment='c')
co2_c = Metabolite(id='co2_c', name='carbon dioxide', formula='CO2', compartment='c')
ru5p__D_c = Metabolite(id='ru5p__D_c', name='ribulose-5-phosphate', formula='C5H11O9P', compartment='c')
xu5p__D_c = cobra.Metabolite(id="xu5p__D_c", name='', formula="C5H10O5P", compartment="c")
r5p_c = cobra.Metabolite(id="r5p_c", name='D-Ribose 5-phosphate', formula="C5H10O5P", compartment="c")
s7p_c = cobra.Metabolite(id="s7p_c", name='Sedoheptulose 7-phosphate', formula="C7H14O7P", compartment="c")
e4p_c = cobra.Metabolite(id="e4p_c", name='D-Erythrose 4-phosphate', formula="C5H11O5P", compartment="c")

model.add_metabolites([glc__D_e, glc__D_c, g6p_c, pep_c, pyr_c, f6p_c, adp_c, atp_c, fdp_c, h_c, dhap_c, g3p_c, h2o_c,
                      _6pgl_c, _6pgc_c, nadp_c, nadph_c, co2_c, ru5p__D_c, xu5p__D_c, r5p_c, s7p_c,  e4p_c,])

In [5]:
# Add RNA Nucleotides as Metabolites

atp_c = Metabolite('atp_c', formula='C10H12N5O13P3', name='ATP', compartment='c')
gtp_c = Metabolite('gtp_c', formula='C10H12N5O14P3', name='GTP', compartment='c')
ctp_c = Metabolite('ctp_c', formula='C9H12N3O14P3', name='CTP', compartment='c')
utp_c = Metabolite('utp_c', formula='C9H11N2O15P3', name='UTP', compartment='c')

model.add_metabolites([atp_c, gtp_c, ctp_c, utp_c,])


In [6]:
# Set bounds
#cobra_config.bounds = -1000, 1000

# Add Reactions
GLCpts = Reaction(id='GLCpts', name='Glucose Phosphate Translocator', subsystem='Pentose Phosphate Pathway', lower_bound=0, upper_bound=1000)
GLCpts.add_metabolites({g6p_c: 1, pep_c: -1, glc__D_e: -1, pyr_c: 1})
GLCpts.gene_reaction_rule = '(b2417 and b1101 and b2415 and b2416) or (b1817 and b1818 and b1819 and b2415 and b2416) or (b2417 and b1621 and b2415 and b2416)'
model.add_reaction(GLCpts)

PGI = Reaction(id='PGI', name='Glucose-6-phosphate Isomerase', lower_bound=-1000, upper_bound=1000)
PGI.add_metabolites({f6p_c: 1, g6p_c: -1})
PGI.gene_reaction_rule = '(b4025)'
model.add_reaction(PGI)

PFK = Reaction(id='PFK', name='Phosphofructokinase', lower_bound=0, upper_bound=1000)
PFK.add_metabolites({adp_c: 1, atp_c: -1, f6p_c: -1, fdp_c: 1, h_c: 1})
PFK.gene_reaction_rule = '(b3916 or b1723)'
model.add_reaction(PFK)

FBA = Reaction(id='FBA', name='Fructose-bisphosphate Aldolase', lower_bound=-1000, upper_bound=1000)
FBA.add_metabolites({dhap_c: 1, fdp_c: -1, g3p_c: 1})
FBA.gene_reaction_rule = '(b2097 or b1773 or b2925)'
model.add_reaction(FBA)

G6PDH2r = Reaction(id='G6PDH2r', name='Glucose-6-phosphate Dehydrogenase', lower_bound=-1000, upper_bound=1000)
G6PDH2r.add_metabolites({_6pgl_c: 1, _6pgc_c: -1, h_c: 1, nadp_c: -1, nadph_c: 1})
G6PDH2r.gene_reaction_rule = '(b1852)'
model.add_reaction(G6PDH2r)

PGL = Reaction(id='PGL', name='6-phosphogluconolactonase', lower_bound=0, upper_bound=1000)
PGL.add_metabolites({_6pgc_c: 1, _6pgl_c: -1, h_c: 1, h2o_c: -1})
PGL.gene_reaction_rule = '(b0767)'
model.add_reaction(PGL)

GND = Reaction(id='GND', name='6-phosphogluconate Dehydrogenase', lower_bound=0, upper_bound=1000)
GND.add_metabolites({_6pgc_c: -1, co2_c: 1, nadp_c: -1, nadph_c: 1, ru5p__D_c: 1})
GND.gene_reaction_rule = '(b2029)'
model.add_reaction(GND)

RPE = Reaction(id='RPE', name='Ribulose-5-phosphate-3-epimerase', lower_bound=-1000, upper_bound=1000)
RPE.add_metabolites({ru5p__D_c: -1, xu5p__D_c: 1})
RPE.gene_reaction_rule = '(b3386 or b4301)'
model.add_reaction(RPE)

RPI = Reaction(id='RPI', name='Ribose-5-phosphate isomerase', lower_bound=-1000, upper_bound=1000)
RPI.add_metabolites({r5p_c: -1, ru5p__D_c: 1})
RPI.gene_reaction_rule = '(b2914 or b4090)'
model.add_reaction(RPI)

TKT1 = Reaction(id='TKT1', name='Transketolase', lower_bound=-1000, upper_bound=1000)
TKT1.add_metabolites({g3p_c: 1, r5p_c: -1, s7p_c: 1, xu5p__D_c: -1})
TKT1.gene_reaction_rule = '(b2935 or b2465)'
model.add_reaction(TKT1)

TALA = Reaction(id='TALA', name='Transaldolase', lower_bound=-1000, upper_bound=1000)
TALA.add_metabolites({e4p_c: 1, f6p_c: 1, g3p_c: -1, s7p_c: -1})
TALA.gene_reaction_rule = '(b2464 or b0008)'
model.add_reaction(TALA)

TKT2 = Reaction(id='TKT2', name='Transketolase', lower_bound=-1000, upper_bound=1000)
TKT2.add_metabolites({e4p_c: -1, f6p_c: 1, g3p_c: 1, xu5p__D_c: -1})
TKT2.gene_reaction_rule = '(b2935 or b2465)'
model.add_reaction(TKT2)




In [7]:
# Add exchange reactions

EX_glc__D_e = Reaction(id='EX_glc__D_e', name='Glucose Exchange', lower_bound=-10, upper_bound=1000)
EX_glc__D_e.add_metabolites({glc__D_e: -1})
model.add_reaction(EX_glc__D_e)

EX_glc__D_c = Reaction(id='EX_glc__D_c', name='Glucose cytosol exchange', lower_bound=-1000, upper_bound=1000)
EX_glc__D_c.add_metabolites({glc__D_e: -1, glc__D_c: 1})
model.add_reaction(EX_glc__D_c)

SK_f6p_c = Reaction(id='SK_f6p_c', name='fructose-6-phosphate sink', lower_bound=0, upper_bound=0)
SK_f6p_c.add_metabolites({f6p_c: -1})
model.add_reaction(SK_f6p_c)

DM_e4p_c = Reaction(id='DM_e4p_c', name='D-Erythrose 4-phosphate sink', lower_bound=-0, upper_bound=1000)
DM_e4p_c.add_metabolites({e4p_c: -1})
model.add_reaction(DM_e4p_c)

EX_pyr_c = Reaction(id='EX_pyr_c', name='pyruvate exchange', lower_bound=-1000, upper_bound=1000)
EX_pyr_c.add_metabolites({pyr_c: -1})
model.add_reaction(EX_pyr_c)

EX_pep_c = Reaction(id='EX_pep_c', name='phosphoenolpyruvate exchange', lower_bound=-1000, upper_bound=1000)
EX_pep_c.add_metabolites({pep_c: -1})
model.add_reaction(EX_pep_c)

EX_adp_c = Reaction(id='EX_adp_c', name='adenosine diphosphate exchange', lower_bound=-1000, upper_bound=1000)
EX_adp_c.add_metabolites({adp_c: -1})
model.add_reaction(EX_adp_c)

EX_atp_c = Reaction(id='EX_atp_c', name='adenosine triphosphate exchange', lower_bound=-1000, upper_bound=1000)
EX_atp_c.add_metabolites({atp_c: -1})
model.add_reaction(EX_atp_c)

EX_h_c = Reaction(id='EX_h_c', name='hydrogen exchange', lower_bound=-1000, upper_bound=1000)
EX_h_c.add_metabolites({h_c: -1})
model.add_reaction(EX_h_c)

EX_h2o_c = Reaction(id='EX_h2o_c', name='water exchange', lower_bound=-1000, upper_bound=1000)
EX_h2o_c.add_metabolites({h2o_c: -1})
model.add_reaction(EX_h2o_c)

EX_co2_c = Reaction(id='EX_co2_c', name='carbon dioxide exchange', lower_bound=-1000, upper_bound=1000)
EX_co2_c.add_metabolites({co2_c: -1})
model.add_reaction(EX_co2_c)

EX_nadph_c = Reaction(id='EX_nadph_c', name='NADPH exchange', lower_bound=-1000, upper_bound=1000)
EX_nadph_c.add_metabolites({nadph_c: -1})
model.add_reaction(EX_nadph_c)

EX_nadp_c = Reaction(id='EX_nadp_c', name='NADP exchange', lower_bound=-1000, upper_bound=1000)
EX_nadp_c.add_metabolites({nadp_c: -1})
model.add_reaction(EX_nadp_c)

EX_dhap_c = Reaction(id='EX_dhap_c', name='dihydroxyacetone phosphate exchange', lower_bound=-1000, upper_bound=1000)
EX_dhap_c.add_metabolites({dhap_c: -1})
model.add_reaction(EX_dhap_c)

# model.add_boundary(model.metabolites.get_by_id("glc__D_e"), type="exchange")
# model.add_boundary(model.metabolites.get_by_id("f6p_c"), type="sink")



Check the model and optimise objective function

In [8]:
# Check that the model is working
print(model.summary)
print(model.reactions)

print(f'{len(model.reactions)} reactions')
print(f'{len(model.metabolites)} metabolites')
print(f'{len(model.genes)} genes')
print("---------")

<bound method Model.summary of <Model toy_model at 0x1e303516aa0>>
[<Reaction GLCpts at 0x1e327313940>, <Reaction PGI at 0x1e327313d00>, <Reaction PFK at 0x1e327313e50>, <Reaction FBA at 0x1e327313910>, <Reaction G6PDH2r at 0x1e32730d540>, <Reaction PGL at 0x1e32730de70>, <Reaction GND at 0x1e32730d900>, <Reaction RPE at 0x1e32730c520>, <Reaction RPI at 0x1e3273139a0>, <Reaction TKT1 at 0x1e303516e90>, <Reaction TALA at 0x1e303516b00>, <Reaction TKT2 at 0x1e3272b6020>, <Reaction EX_glc__D_e at 0x1e3272e1990>, <Reaction EX_glc__D_c at 0x1e3272e1930>, <Reaction SK_f6p_c at 0x1e3272e0490>, <Reaction DM_e4p_c at 0x1e3272e2350>, <Reaction EX_pyr_c at 0x1e3272e28f0>, <Reaction EX_pep_c at 0x1e3272e15d0>, <Reaction EX_adp_c at 0x1e3272e06a0>, <Reaction EX_atp_c at 0x1e3272e08e0>, <Reaction EX_h_c at 0x1e3272e1c60>, <Reaction EX_h2o_c at 0x1e3272e0a00>, <Reaction EX_co2_c at 0x1e3272e0a30>, <Reaction EX_nadph_c at 0x1e3272e09a0>, <Reaction EX_nadp_c at 0x1e32730e770>, <Reaction EX_dhap_c at 0x

In [9]:
print("Reactions")
print("---------")
for x in model.reactions:
    print("%s : %s" % (x.id, x.reaction))

print("")
print("Metabolites")
print("-----------")
for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

print("")
print("Genes")
print("-----")
for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))

Reactions
---------
GLCpts : glc__D_e + pep_c --> g6p_c + pyr_c
PGI : g6p_c <=> f6p_c
PFK : atp_c + f6p_c --> adp_c + fdp_c + h_c
FBA : fdp_c <=> dhap_c + g3p_c
G6PDH2r : 6pgc_c + nadp_c <=> 6pgl_c + h_c + nadph_c
PGL : 6pgl_c + h2o_c --> 6pgc_c + h_c
GND : 6pgc_c + nadp_c --> co2_c + nadph_c + ru5p__D_c
RPE : ru5p__D_c <=> xu5p__D_c
RPI : r5p_c <=> ru5p__D_c
TKT1 : r5p_c + xu5p__D_c <=> g3p_c + s7p_c
TALA : g3p_c + s7p_c <=> e4p_c + f6p_c
TKT2 : e4p_c + xu5p__D_c <=> f6p_c + g3p_c
EX_glc__D_e : glc__D_e <=> 
EX_glc__D_c : glc__D_e <=> glc__D_c
SK_f6p_c : f6p_c --> 
DM_e4p_c : e4p_c --> 
EX_pyr_c : pyr_c <=> 
EX_pep_c : pep_c <=> 
EX_adp_c : adp_c <=> 
EX_atp_c : atp_c <=> 
EX_h_c : h_c <=> 
EX_h2o_c : h2o_c <=> 
EX_co2_c : co2_c <=> 
EX_nadph_c : nadph_c <=> 
EX_nadp_c : nadp_c <=> 
EX_dhap_c : dhap_c <=> 

Metabolites
-----------
 glc__D_e : C6H12O6
 glc__D_c : C6H12O6
    g6p_c : C6H11O9P
    pep_c : C3H2O6P
    pyr_c : C3H4O3
    f6p_c : C6H11O9P
    adp_c : C10H15N5O10P2
    atp_c

In [10]:
# Set the objective for the model

obj = 'DM_e4p_c' # change to e4p_c

model.objective = obj

print(model.objective.expression)
print(model.objective.direction)

print(model.summary())

solution = model.optimize()
print(solution)
print(solution.fluxes[obj])

# print(model.reactions.EX_glc__D_e)

flux_variability_analysis(model, model.reactions[:])


1.0*DM_e4p_c - 1.0*DM_e4p_c_reverse_1a776
max
Objective
1.0 DM_e4p_c = 10.0

Uptake
------
Metabolite    Reaction  Flux  C-Number C-Flux
     atp_c    EX_atp_c 6.667        10 42.55%
  glc__D_e EX_glc__D_e    10         6 38.30%
     pep_c    EX_pep_c    10         3 19.15%

Secretion
---------
Metabolite  Reaction   Flux  C-Number C-Flux
     e4p_c  DM_e4p_c    -10         5 30.00%
     adp_c  EX_adp_c -6.667        10 40.00%
    dhap_c EX_dhap_c -6.667         3 12.00%
       h_c    EX_h_c -6.667         0  0.00%
     pyr_c  EX_pyr_c    -10         3 18.00%

<Solution 10.000 at 0x1e32744d5a0>
10.000000000000002


,minimum,maximum
GLCpts,10.000000,10.000000
PGI,10.000000,10.000000
PFK,6.666667,6.666667
FBA,6.666667,6.666667
G6PDH2r,0.000000,496.666667
PGL,0.000000,496.666667
GND,0.000000,0.000000
RPE,-3.333333,-3.333333
RPI,-3.333333,-3.333333
TKT1,3.333333,3.333333


Write and save files

In [11]:
cobra.io.save_json_model(model, model_json_file)

In [12]:
cobra.io.write_sbml_model(model, model_xml_file)